In [19]:
#Importing Necessary Pacakges
import tensorflow as tf
from tensorflow import keras

In [3]:
#Importing Data Set
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) # ylabels are oh-encoded
n_train = mnist.train.num_examples # 55,000
n_validation = mnist.validation.num_examples # 5000
n_test = mnist.test.num_examples # 10,000

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#Defining Neural Network Architecture
n_input = 784 # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10 # output layer (0-9 digits)

In [5]:
#Defining Hyper Parameters
learning_rate = 1e-4 #The learning rate represents how much the parameters will adjust at each step of the learning process
n_iterations = 1000 #The number of iterations refers to how many times we go  through the training step
batch_size = 128 #Batch size refers to how many training examples we are using at each step
dropout = 0.5 #The dropout variable represents a threshold at which we eliminate some units at random

In [6]:
#Building Tensor Flow Graph -- Defining Tensors
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

In [7]:
#Initializing Weights
weights = {
'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1],
stddev=0.1)),
'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2],
stddev=0.1)),
'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3],
stddev=0.1)),
'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output],
stddev=0.1)),
}

In [8]:
#Initializing Biases
biases = {
'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [9]:
#Setting up Layers of the Network
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

W0913 13:12:26.563478  9912 deprecation.py:506] From <ipython-input-9-708ee5975bca>:5: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
#Defining A Loss Function To Optimize Weights and Biases
cross_entropy = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits(
labels=Y, logits=output_layer
))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

W0913 13:13:55.873913  9912 deprecation.py:323] From <ipython-input-10-d87742589845>:4: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
#Defining Accuracy 
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [18]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={
    X: batch_x, Y: batch_y, keep_prob: dropout
})

In [20]:
# print loss and accuracy (per minibatch)
if i % 100 == 0:
    minibatch_loss, minibatch_accuracy = sess.run(
        [cross_entropy, accuracy],
        feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0}
        )
    print(
    "Iteration",
    str(i),
    "\t| Loss =",
    str(minibatch_loss),
    "\t| Accuracy =",
    str(minibatch_accuracy)
    )

In [17]:
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y:
mnist.test.labels, keep_prob: 1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.9212
